<a href="https://colab.research.google.com/github/Subramanian1990/Deep-Learning/blob/master/LargeCNN_practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import copy

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

In [5]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True, 
                                        transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                        download=True, 
                                        transform=transform_test)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
num_classes = 10

In [7]:
batch_size = 16
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)

print(images[1].shape)
print(labels[1].item())

torch.Size([16, 3, 224, 224])
torch.Size([3, 224, 224])
8


In [0]:
def evaluation(dataset, model):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

def vgg(dataset):
  vgg = models.vgg16_bn(pretrained=True)
  final_in_features = vgg.classifier[6].in_features
  mod_classifier = list(vgg.classifier.children())[:-1]
  mod_classifier.extend([nn.Linear(final_in_features, num_classes)])
  
  vgg.classifier = nn.Sequential(*mod_classifier)
  

  
  for param in vgg.parameters():
    param.requires_grad = False
  final_in_features = vgg.classifier[6].in_features
  vgg.classifier[6] = nn.Linear(final_in_features, num_classes)

  vgg = vgg.to(device)
  loss_fn = nn.CrossEntropyLoss()
  opt = optim.SGD(vgg.parameters(), lr=0.05)
  loss_epoch_arr = []
  max_epochs = 1

  min_loss = 1000

  n_iters = np.ceil(50000/batch_size)
  loss_epoch_arr = []
  max_epochs = 1
  min_loss = 1000

  n_iters = np.ceil(50000/batch_size)

  for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = vgg(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
        if min_loss > loss.item():
            min_loss = loss.item()
            best_model = copy.deepcopy(vgg.state_dict())
            print('Min loss %0.2f' % min_loss)
        
        if i % 100 == 0:
            print('Iteration: %d/%d, Loss: %0.2f' % (i, n_iters, loss.item()))
            
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
    loss_epoch_arr.append(loss.item())
    




    # print(evaluation(trainloader, vgg), evaluation(testloader, vgg))


In [10]:
vgg(trainloader)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth


Min loss 2.36
Iteration: 0/3125, Loss: 2.36
Min loss 2.35
Min loss 2.16
Min loss 1.86
Min loss 1.73
Min loss 1.62
Min loss 1.54
Min loss 1.26
Min loss 1.22
Iteration: 100/3125, Loss: 2.10
Min loss 1.09
Min loss 0.96
Min loss 0.81
Iteration: 200/3125, Loss: 1.89
Iteration: 300/3125, Loss: 1.05
Iteration: 400/3125, Loss: 2.42
Min loss 0.75
Iteration: 500/3125, Loss: 1.78
Min loss 0.69
Iteration: 600/3125, Loss: 1.48
Iteration: 700/3125, Loss: 1.75
Min loss 0.61
Iteration: 800/3125, Loss: 2.01
Iteration: 900/3125, Loss: 2.53
Iteration: 1000/3125, Loss: 2.05
Iteration: 1100/3125, Loss: 0.70
Iteration: 1200/3125, Loss: 2.77
Iteration: 1300/3125, Loss: 1.27
Iteration: 1400/3125, Loss: 2.61
Iteration: 1500/3125, Loss: 3.05
Iteration: 1600/3125, Loss: 1.51
Iteration: 1700/3125, Loss: 1.54
Iteration: 1800/3125, Loss: 1.89
Min loss 0.51
Iteration: 1900/3125, Loss: 1.99
Iteration: 2000/3125, Loss: 2.97
Iteration: 2100/3125, Loss: 1.95
Iteration: 2200/3125, Loss: 1.94
Iteration: 2300/3125, Loss: 1

In [0]:
def resnet(dataset):
  resnet = models.resnet18(pretrained=True)
  for param in resnet.parameters():
    param.requires_grad = False
  in_features = resnet.fc.in_features
  resnet.fc = nn.Linear(in_features, num_classes)
  for param in resnet.parameters():
    if param.requires_grad:
        print(param.shape)
  resnet = resnet.to(device)
  loss_fn = nn.CrossEntropyLoss()
  opt = optim.SGD(resnet.parameters(), lr=0.01)    
  loss_epoch_arr = []
  max_epochs = 4
 
  min_loss = 1000

  n_iters = np.ceil(50000/batch_size)

  for epoch in range(max_epochs):

    for i, data in enumerate(dataset, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = resnet(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
        if min_loss > loss.item():
            min_loss = loss.item()
            best_model = copy.deepcopy(resnet.state_dict())
            print('Min loss %0.2f' % min_loss)
        
        if i % 100 == 0:
            print('Iteration: %d/%d, Loss: %0.2f' % (i, n_iters, loss.item()))
            
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
    loss_epoch_arr.append(loss.item())
        
    # print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
    #     epoch, max_epochs, 
    #     evaluation(testloader, resnet), evaluation(trainloader, resnet)))
    
    
  plt.plot(loss_epoch_arr)
  plt.show()  
    

In [0]:
resnet(trainloader)

torch.Size([10, 512])
torch.Size([10])
Min loss 2.63
Iteration: 0/3125, Loss: 2.63
Min loss 2.36
Min loss 2.28
Min loss 2.18
Min loss 2.13
Min loss 2.06
Min loss 2.06
Min loss 1.88
Min loss 1.80
Min loss 1.67
Min loss 1.65
Min loss 1.63
Iteration: 100/3125, Loss: 1.99
Min loss 1.60
Min loss 1.58
Min loss 1.55
Min loss 1.53
Min loss 1.50
Min loss 1.43
Min loss 1.34
Min loss 1.30
Iteration: 200/3125, Loss: 1.44
Min loss 1.26
Min loss 1.22
Min loss 1.17
Min loss 1.03
Iteration: 300/3125, Loss: 1.40
Min loss 0.99
Iteration: 400/3125, Loss: 1.35
Min loss 0.85
Iteration: 500/3125, Loss: 1.38
Min loss 0.84
Iteration: 600/3125, Loss: 1.11
Iteration: 700/3125, Loss: 1.21
Min loss 0.77
Iteration: 800/3125, Loss: 1.48
Min loss 0.65
Iteration: 900/3125, Loss: 1.30
Iteration: 1000/3125, Loss: 1.34
Iteration: 1100/3125, Loss: 1.19
Iteration: 1200/3125, Loss: 1.24
Iteration: 1300/3125, Loss: 1.37
Iteration: 1400/3125, Loss: 1.34
Iteration: 1500/3125, Loss: 1.47
Iteration: 1600/3125, Loss: 1.18
Iterat

In [0]:
def inception(dataset):
  inception = models.inception_v3(pretrained=True)
  for param in inception.parameters():
    param.requires_grad = False
  aux_in_features = inception.AuxLogits.fc.in_features
  inception.AuxLogits.fc = nn.Linear(aux_in_features, num_classes)
  
  in_features = inception.fc.in_features
  inception.fc = nn.Linear(in_features, num_classes)
  for param in inception.parameters():
    if param.requires_grad:
        print(param.shape)

  for param in inception.parameters():
    if param.requires_grad:
        print(param.shape)
  transform_train = transforms.Compose([
    transforms.RandomResizedCrop(299), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

  transform_test = transforms.Compose([
    transforms.RandomResizedCrop(299), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])      
  
  inception = inception.to(device)
  loss_fn = nn.CrossEntropyLoss()
  opt = optim.SGD(inception.parameters(), lr=0.01)

  loss_epoch_arr = []
  max_epochs = 1

  min_loss = 1000

  n_iters = np.ceil(50000/batch_size)

  for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs, aux_outputs = inception(inputs)
        loss = loss_fn(outputs, labels) + 0.3 * loss_fn(aux_outputs, labels)
        loss.backward()
        opt.step()
        
        if min_loss > loss.item():
            min_loss = loss.item()
            best_model = copy.deepcopy(inception.state_dict())
            print('Min loss %0.2f' % min_loss)
        
        if i % 100 == 0:
            print('Iteration: %d/%d, Loss: %0.2f' % (i, n_iters, loss.item()))
            
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
    loss_epoch_arr.append(loss.item())
        
    print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
        epoch, max_epochs, 
        evaluation_inception(testloader, inception), 
        evaluation_inception(trainloader, inception)))
    
    
  plt.plot(loss_epoch_arr)
  plt.show()